## PDI - Rebalancing data for September 2023

## Configuring packages

In [3]:
import sys 
sys.path.append('../')
import pandas as pd
import numpy as np
import datetime
from matplotlib import pyplot as plt
import time
import requests
import decouple
import db_funcs
from web3 import Web3
from abis import index_anatomy, univ3_pool , erc20_contract, index_registry


infura_url = decouple.config("ETHEREUM_INFURA_URL")
web3_object = Web3(Web3.HTTPProvider(infura_url))

In [4]:
from pycoingecko import CoinGeckoAPI
key =  decouple.config("CG_KEY")
cg = CoinGeckoAPI(api_key=key)

In [5]:
from defillama import DefiLlama
llama = DefiLlama()

In [6]:
import plotly.express as px
import plotly.graph_objects as go

In [7]:
from plotly.offline import plot, iplot, init_notebook_mode
init_notebook_mode(connected=True)

## Index Variables and settings

In [8]:
db_key_date = '2023-09-01'
db_liquidity_table = 'pdi_liquidities'
db_benchmark_table = 'pdi_benchmark_data'
min_weight = 0.01
max_weight = 0.3
supply_threshold = 0.3
twap_liq_threshold = 1e6
liquidity_consistency = 90 # In days
liveness_threshold = 365 # In days

In [9]:
liq = 5*10**6
liq_os = liq / 2 # one-side liquidity 
trade_value_tiny = 1e2
trade_value = 1e5
max_slippage = (liq_os + trade_value_tiny) / (liq_os + trade_value) - 1
# round to the nearest .5
max_slippage = round(2 * max_slippage, 2) / 2 
max_slippage

-0.04

## Data retrieving and filtering

### Token inclusion criteria

1. **Project and token characteristics**
    1. The project must have a listing on Defi Llama.
    1. The project’s token should have been listed on CoinGecko with pricing data at least 1 year prior to the date of inclusion in the index. 
    1. The project should have a token that is native to Ethereum L1 or L2. This excludes wrapped variants, where the underlying tokens are locked on an alt-L1.
    1. The project should be a going concern, with a dedicated team actively building, supporting and maintaining the project. 
    1. No rebasing or deflationary tokens.
    1. The project must be widely considered to be building a useful protocol or product. 
    1. Projects that have circular feedback characteristics at the core of their offering will not be considered.
    1. Synthetic tokens which derive their value from external price feeds are not permissible. 
    1. The project’s token must not have the ability to pause token transfers.
    1. The project’s protocol or product must have significant usage.
   <br>
1. **Pricing requirmeents**
    1. Token should have a manipulation resistant price feed in the form of a Chainlink price feed or a Uniswap V3 TWAP oracle with a cardinality of at least 300 and a minimum liquidity level of $1mm.
    <br>
1. **Token Supply requirements**
    1. The project's token must have a circulating supply greater than 30% of the max supply. In cases where a token does not have a max supply, the minting mechanics would need to be assessed.
    <br>
1. **Liquidity Requirements**
    1. The token must be listed on a supported exchange.
    1. The token should have in aggregate at least \$5mm of onchain liquidity across Uniswap v2, Uniswap v3, Sushiswap, Balancer v1, Balancer v2, Bancor v2 and Bancor v3.
    1. The price should experience no more than 4% price impact when executing a $100k trade.
    1. The token must have shown consistent DeFi liquidity on Ethereum.
    <br>
1. **Security Requirements**
    1. The project must have been audited by smart contract security professionals with the audit report(s) publicly available. Alternatively, the protocol must have been operating long enough to create a consensus about its safety in the decentralised finance community.

### Tokens list

In [10]:
def normalize_symb(symb):
    return ''.join(c for c in symb if c.isalnum()).upper()

#### DefiLlama

In [11]:
protocols = llama.get_all_protocols()

In [12]:
top_defi_tokens = {} 
for pr in protocols[:1000]:
    if 'Ethereum' in pr['chains'] and pr['address'] and pr['address']!='-' and pr['gecko_id'] != '-':
        symbol = normalize_symb(pr['symbol']) 
        address = pr['address'].lower().split(":")[-1]
        top_defi_tokens[symbol] = {
            'symbol': symbol, 
            'address': address 
        }

# Switching NXM for WNXM

del top_defi_tokens['NXM']
top_defi_tokens['WNXM'] = {
    'symbol' : 'WNXM',
    'address' : '0x0d438f3b5175bebc262bf23753c1e53d03432bde'
}

#### CoinGecko

##### Tokens addresses

In [13]:
coins_list = pd.DataFrame(cg.get_coins_list(include_platform=True))

In [14]:
tokens = {} 
tokens_by_addr = {}
for index, coin in coins_list.iterrows():
    if len(coin['platforms']) >= 1 and 'ethereum' in list(coin['platforms'].keys()):
        symbol = normalize_symb(coin['symbol'])
        address = coin['platforms']['ethereum'].lower().split(":")[-1]
        token = {
            'symbol': symbol,
            'address': address 
        }
        tokens[symbol] = token
        tokens_by_addr[address] = token



In [15]:
# These token's addresses cant be found in the coingecko list of eth addresses
exclude_symbols = []
for symb, token in top_defi_tokens.items():
    if token['address'] not in tokens_by_addr.keys():
        exclude_symbols.append(token['symbol'])


In [16]:
for symb in exclude_symbols:
    del top_defi_tokens[symb]

top_defi_token_symbols = list(top_defi_tokens.keys())


##### Market data

In [17]:
cg_tokens_data = cg.get_coins_markets(vs_currency='USD', order='market_cap_desc')
for i in range(2, 20):
    cg_tokens_data.extend(cg.get_coins_markets(vs_currency='USD', order='market_cap_desc', page=i))
    
cg_tokens_data = pd.DataFrame.from_records(cg_tokens_data)
cg_tokens_data.set_index(cg_tokens_data['symbol'].str.upper(),inplace=True)

In [18]:
tokens_market_data = cg_tokens_data.set_index(cg_tokens_data['symbol'].str.upper())


In [19]:
duplicated = tokens_market_data.index[tokens_market_data.index.duplicated(keep='first')]
tokens_market_data = tokens_market_data.drop(index=duplicated)

In [20]:
tokens_market_data.query('index in @top_defi_token_symbols',inplace=True)
tokens_market_data.set_index('id',inplace=True)
tokens_market_data = tokens_market_data[['symbol','name','current_price','market_cap','market_cap_rank','fully_diluted_valuation','circulating_supply','total_supply','max_supply']]

### Token's supply check 

In [21]:

sp_passed_index = (tokens_market_data['circulating_supply'] / tokens_market_data['total_supply']) > supply_threshold
sp_passed = tokens_market_data[sp_passed_index]

### Retrieve historical data & livetime check

In [22]:
exclude_list = []

prices_data = pd.DataFrame()
marketcaps = pd.DataFrame() 
for id, data in sp_passed.iterrows():
    data = cg.get_coin_market_chart_by_id(id, vs_currency='USD', days='max')
    df_prices = pd.DataFrame(data['prices'], columns=['date', id])
    df_prices = df_prices[df_prices[id] > 0]
    df_prices['date'] = pd.to_datetime(df_prices['date'], unit='ms').dt.date
    df_prices['date'] = pd.to_datetime(df_prices['date'])
    df_prices = df_prices.set_index('date', drop=True)
    df_prices = df_prices.loc[~df_prices.index.duplicated(keep='first')]
    
    if len(df_prices) < liveness_threshold:
        print(f'Excluding {id}, prices data available only for {len(df_prices)} < {liveness_threshold} days')
        exclude_list.append(id)
        continue
    prices_data = pd.concat([prices_data, df_prices], axis=1)
    
    df_mcaps = pd.DataFrame(data['market_caps'], columns=['date', id])
    df_mcaps = df_mcaps[df_mcaps[id] > 0]
    df_mcaps['date'] = pd.to_datetime(df_mcaps['date'], unit='ms').dt.date
    df_mcaps['date'] = pd.to_datetime(df_mcaps['date'])
    df_mcaps = df_mcaps.set_index('date', drop=True)
    df_mcaps = df_mcaps.loc[~df_mcaps.index.duplicated(keep='first')]
    
    if len(df_mcaps) < liveness_threshold:
        print(f'Note: {id}, marketcap data available only for {len(df_mcaps)} < {liveness_threshold} days')
        ## continue
    df_mcaps = df_mcaps.reindex(df_prices.index)
    marketcaps = pd.concat([marketcaps, df_mcaps], axis=1)
    

Excluding short-term-t-bill-token, prices data available only for 10 < 365 days
Note: centrifuge, marketcap data available only for 360 < 365 days
Excluding smardex, prices data available only for 170 < 365 days
Note: sommelier, marketcap data available only for 181 < 365 days
Excluding grai, prices data available only for 108 < 365 days
Excluding boosted-lusd, prices data available only for 277 < 365 days
Note: conic-finance, marketcap data available only for 229 < 365 days
Note: bumper, marketcap data available only for 339 < 365 days
Excluding sudoswap, prices data available only for 197 < 365 days
Excluding alongside-crypto-market-index, prices data available only for 234 < 365 days
Excluding unsheth, prices data available only for 181 < 365 days


### Non quantative checks

In [23]:
manual_exclusions = [
    {
        'gecko_id': 'wrapped-bitcoin',
        'reason': 'Not DeFi'
    },
    {
        'gecko_id': 'weth',
        'reason': 'Not DeFi'
    },
    {
        'gecko_id': 'ethernity-chain',
        'reason': 'Not DeFi'
    },
    {
        'gecko_id': 'nftx',
        'reason': 'Not DeFi'
    },
    {
        'gecko_id': 'rari-governance-token',
        'reason': 'Deprecated asset'
    },
    {
        'gecko_id': 'superfarm',
        'reason': 'Not DeFi'
    },
    {
        'gecko_id': 'barnbridge',
        'reason': 'Criteria 1.I'
    },
    {
        'gecko_id': 'usd-coin',
        'reason': 'stable'
    },
    {
        'gecko_id': 'fei-usd',
        'reason': 'stable'
    },
    {
        'gecko_id': 'synapse-2',
        'reason': 'bridge'
    },
    {
        'gecko_id': 'huobi-btc',
        'reason': 'Not DeFi'
    },
    {
        'gecko_id': 'bone-shibaswap',
        'reason': 'Criteria 1.6'
    },
    {
        'gecko_id': 'crypto-com-chain',
        'reason': 'Not DeFi'
    },
    {
        'gecko_id': 'woo-network',
        'reason': 'Criteria 2.1'
    },
    {
        'gecko_id': 'immutable-x',
        'reason': 'Not DeFi'
    },
    {
        'gecko_id':'injective-protocol',
        'reason': '1.3 - Not native to Ethereum mainnet'
    }


]
for token in manual_exclusions:
    exclude_list.append(token['gecko_id'])


In [24]:
passed_q_and_non_q_checks = sp_passed.query('index not in @exclude_list' )
passed_q_and_non_q_checks

,symbol,name,current_price,market_cap,market_cap_rank,fully_diluted_valuation,circulating_supply,total_supply,max_supply
id,,,,,,,,,
uniswap,uni,Uniswap,4.370000,3293905588,23,4.369927e+09,7.537667e+08,1.000000e+09,1.000000e+09
lido-dao,ldo,Lido DAO,1.520000,1354026062,36,1.525284e+09,8.877207e+08,1.000000e+09,1.000000e+09
maker,mkr,Maker,1132.070000,1022351408,43,1.140620e+09,9.013109e+05,9.776310e+05,1.005577e+06
aave,aave,Aave,54.850000,797024618,47,8.777427e+08,1.452862e+07,1.600000e+07,1.600000e+07
havven,snx,Synthetix Network,1.980000,638749068,57,6.401816e+08,3.227828e+08,3.235067e+08,3.235067e+08
...,...,...,...,...,...,...,...,...,...
meta,mta,mStable Governance: Meta,0.032156,1595805,1678,3.194599e+06,4.995323e+07,9.596498e+07,1.000000e+08
dfx-finance,dfx,DFX Finance,0.031995,1337240,1775,3.209649e+06,4.166314e+07,1.000000e+08,1.000000e+08
pickle-finance,pickle,Pickle Finance,0.898039,1191034,1836,2.906151e+06,1.315013e+06,3.208664e+06,3.208664e+06


### Liquidity check

To derive a slippage threshold, consider an abstract univ2-like pool with $5m\$$ overall liquidity. 
I.e if pool has reserves $R_1$, $R_2$ and $p_1$ and $p_2$ are the correspondent tokens prices (in USD), then
$$R_1p_1 = R_2p_2 = 2.5m \$$$

Swapping $x$ token1 for token2 will result in the output amount $y$ such that
$$R_1\cdot R_2 = (R_1 + x) \cdot (R_2 - y)$$
$$y = \frac{R_2\cdot x}{R_1 + x}$$
Then the delivery price will be
$$p_d = \frac{y}{x} = \frac{R_2}{R_1 + x}$$
The spot price may be estimated as a delivery price for relatively small input amount $x_0$
$$p_s = \frac{y_0}{x_0} = \frac{R_2}{R_1 + x_0}$$

Then the theoretical slippage for this pool will be
$$slippage = \frac{p_d}{p_s} - 1 = \frac{R_1+x_0}{R_1+x} - 1 = \frac{p_1R_1+p_1x_0}{p_1R_1+p_1x}-1$$

In [25]:
url_0x = "https://api.0x.org/swap/v1/quote" 
header = {'0x-api-key': decouple.config("ZEROEX_KEY")}

In [26]:
decimals = 18
slippages = []
for id, data in passed_q_and_non_q_checks.iterrows():
    symbol = data['symbol'].upper()
    try:
        query = {
            'buyToken': tokens[symbol]['address'],
            'sellToken': 'ETH',
            'sellAmount': int(trade_value_tiny / float(cg_tokens_data[cg_tokens_data['symbol'] == 'eth']['current_price']) * 10 ** decimals),
            'enableSlippageProtection':'true'
        }
        
        # spot price is calculated as a price for 100$ swap
        resp = requests.get(url_0x, params=query, headers=header)
        time.sleep(1)
        swap = resp.json()
        spot_price = float(swap['price'])
        
        query['sellAmount'] = int(trade_value / float(cg_tokens_data[cg_tokens_data['symbol'] == 'eth']['current_price']) * 10 ** decimals)
        resp = requests.get(url_0x, params=query,headers=header)
        swap = resp.json()
        time.sleep(1)
        del_price = float(swap['price'])
        
        slippage = del_price / spot_price - 1   
        slippages.append({
            'id': id,
            'spot_price': spot_price,
            'delivery_price': del_price,
            'slippage': slippage
        })
    except KeyError:
        print(id)
        continue
    

/var/folders/4t/6j5ypx_s6r771shjr5dcct5w0000gn/T/ipykernel_36406/2452686901.py:9: FutureWarning:

Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead

/var/folders/4t/6j5ypx_s6r771shjr5dcct5w0000gn/T/ipykernel_36406/2452686901.py:19: FutureWarning:

Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead



centrifuge
nest
umee
mimo-parallel-governance-token


In [27]:
slippages_df = pd.DataFrame(slippages)
slippages_df.head(25).sort_values('slippage',ascending=False).reset_index(drop=True)

,id,spot_price,delivery_price,slippage
0,uniswap,3.713004e+02,3.710767e+02,-0.000603
1,maker,1.436598e+00,1.432915e+00,-0.002564
2,aave,2.975616e+01,2.960122e+01,-0.005207
3,rocket-pool,7.379681e+01,7.323727e+01,-0.007582
4,convex-finance,6.065409e+02,6.016488e+02,-0.008066
5,lido-dao,1.078105e+03,1.065863e+03,-0.011355
6,wrapped-nxm,5.470634e+01,5.408466e+01,-0.011364
7,curve-dao-token,3.735826e+03,3.689793e+03,-0.012322
8,yearn-finance,2.994750e-01,2.954578e-01,-0.013414
9,bancor,3.669221e+03,3.605234e+03,-0.017439


In [28]:
# Writing to database
slippages_to_save = slippages_df[['id','slippage']].head(50)
asset_columns = db_funcs.convert_to_sql_strings(list(slippages_to_save['id']))
slippage_values = list(slippages_to_save['slippage'])
db_funcs.insert_values(db_key_date,asset_columns,slippage_values,db_liquidity_table)

In [29]:
def check_avg_liquidity(date_delta):
    start_date = str(datetime.date.fromisoformat(db_key_date) - datetime.timedelta(days=date_delta))
    dataframe = pd.read_sql(f'Select * from {db_liquidity_table} where date >= ? ',db_funcs.create_connection(db_funcs.db),index_col='date',parse_dates=['date'],params=[start_date])
    dataframe = dataframe.ewm(span=3).mean().iloc[-1]
    dataframe = dataframe[dataframe > max_slippage]
    dataframe.index = db_funcs.convert_from_sql_strings(list(dataframe.index))
    return dataframe, list(dataframe.index)

avg_liq_dataframe ,include_list = check_avg_liquidity(liquidity_consistency)

avg_liq_dataframe

uniswap                     -0.000225
lido-dao                    -0.006679
aave                        -0.002964
rocket-pool                 -0.005889
curve-dao-token             -0.013426
frax-share                  -0.019358
maker                       -0.004804
convex-finance              -0.009835
compound-governance-token   -0.019427
injective-protocol          -0.035517
balancer                    -0.024742
yearn-finance               -0.013775
bancor                      -0.019294
badger-dao                  -0.018450
wrapped-nxm                 -0.011617
alchemix                    -0.034463
Name: 2023-09-01 00:00:00, dtype: float64

In [30]:
liquidity_check = passed_q_and_non_q_checks.query('index in @include_list')
qualified_assets = liquidity_check.query('index not in @exclude_list')
qualified_assets

,symbol,name,current_price,market_cap,market_cap_rank,fully_diluted_valuation,circulating_supply,total_supply,max_supply
id,,,,,,,,,
uniswap,uni,Uniswap,4.370000,3293905588,23,4.369927e+09,7.537667e+08,1.000000e+09,1.000000e+09
lido-dao,ldo,Lido DAO,1.520000,1354026062,36,1.525284e+09,8.877207e+08,1.000000e+09,1.000000e+09
maker,mkr,Maker,1132.070000,1022351408,43,1.140620e+09,9.013109e+05,9.776310e+05,1.005577e+06
aave,aave,Aave,54.850000,797024618,47,8.777427e+08,1.452862e+07,1.600000e+07,1.600000e+07
rocket-pool,rpl,Rocket Pool,22.010000,433518316,83,4.335183e+08,1.969437e+07,1.969437e+07,NaN
frax-share,fxs,Frax Share,5.670000,416017054,87,5.651963e+08,7.337133e+07,9.968150e+07,9.968150e+07
curve-dao-token,crv,Curve DAO,0.436222,379830193,94,1.441050e+09,8.706090e+08,2.005113e+09,3.303030e+09
compound-governance-token,comp,Compound,40.340000,276488863,114,4.032751e+08,6.856086e+06,1.000000e+07,1.000000e+07
convex-finance,cvx,Convex Finance,2.680000,214786154,143,2.681895e+08,8.008745e+07,9.885026e+07,1.000000e+08


### Marketcap ranking & filtering

In [31]:
chosen_tokens = qualified_assets.sort_values("market_cap", ascending=False)

Check for TWAP liquidity

In [32]:
# UNIv3 TWAP pools
current_twap_pools = [
    {
    'gecko_id': 'ribbon-finance',
    'pool address': '0xfe0df74636bc25c7f2400f22fe7dae32d39443d2'
    },
    {
    'gecko_id': "rocket-pool",
    'pool address': '0xe42318eA3b998e8355a3Da364EB9D48eC725Eb45'
    }
]

def get_pool_liquidity(pool_dict):
    pool_contract = web3_object.eth.contract(address= web3_object.to_checksum_address(pool_dict['pool address']),abi= univ3_pool)
    token_0_address = pool_contract.functions.token0().call()
    token_1_address = pool_contract.functions.token1().call()
    token_0 = web3_object.eth.contract(address= token_0_address,abi=erc20_contract)
    token_1 = web3_object.eth.contract(address=token_1_address,abi = erc20_contract)
    token_0_balance = token_0.functions.balanceOf(web3_object.to_checksum_address(pool_dict['pool address'])).call() 
    token_1_balance = token_1.functions.balanceOf(web3_object.to_checksum_address(pool_dict['pool address'])).call()
    pool_liquidity = token_0_balance/(10**token_0.functions.decimals().call()) * float(cg.get_coin_info_from_contract_address_by_id('ethereum',token_0_address)['market_data']['current_price']['usd']) + token_1_balance/(10**token_1.functions.decimals().call()) * float(cg.get_coin_info_from_contract_address_by_id('ethereum',token_1_address)['market_data']['current_price']['usd'])
    return pool_liquidity

twap_warnings = []

for pool in current_twap_pools:
    if pool['gecko_id'] in chosen_tokens.index and get_pool_liquidity(pool) < twap_liq_threshold:
        twap_warnings.append((pool['gecko_id'],get_pool_liquidity(pool)))

print(f" Removing the following under funded twap pools: {twap_warnings}")

chosen_tokens =  chosen_tokens.query('index not in @twap_warnings')


 Removing the following under funded twap pools: []


In [33]:
chosen_tokens_ids = list(chosen_tokens.index)
prices_data = prices_data[chosen_tokens_ids]
marketcaps = marketcaps[chosen_tokens_ids]

In [34]:
without_nan_index = (marketcaps.isnull().sum(axis=1) == 0) & (prices_data.isnull().sum(axis=1) == 0)
marketcaps = marketcaps[without_nan_index]
prices_data = prices_data[without_nan_index]

## Weighting 

**Weighting requirements**
* The maximum weight any one token can have is 30%.
* All excess weight is proportionally redistributed to all uncapped tokens. After this has been completed, if another token now exceeds the 30% threshold the excess will be redistributed to the remaining uncapped tokens. This process will occur iteratively until there are no tokens that exceed the maximum weight.
* Any asset with a weight below 1% will be removed from the index.


### Weight mcaps

In [35]:
weights = marketcaps.div(marketcaps.sum(axis=1), axis=0)
weights = weights.sort_values(weights.last_valid_index(), axis=1, ascending=False)

In [36]:
fig = px.line(weights,
              labels={'value': 'weight, %', 'variable': ''})
fig.update_traces(
    hovertemplate="%{y}"
)
fig.update_yaxes(
    tickformat=".2%",
)
fig.update_xaxes(
    showspikes=True,
    spikethickness=2,
    spikedash="dot",
    spikecolor="#999999",
    spikemode="across",
)
fig.update_layout(
    ## showlegend=False,
    hovermode="x",
    hoverdistance=100,  ## Distance to show hover label of data point
    spikedistance=1000,  ## Distance to show spike
    template='plotly_white',
    title='Weights without max constraint'
)

/Users/olivermehr/.local/share/virtualenvs/rebalancing-data-VGaZtODH/lib/python3.11/site-packages/_plotly_utils/basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



### Adjust weights

This process adjusts weights to adhere to the weight constraints described in our methodology. In addition the `remove_tiny_weights` function checks for any assets with a weight below the minimum threshold of $0.5\%$ and proportionally increases all remaining weights (which are $> 0.5\%$). Note that after this procedure readjusting weights to respect maximum may be needed.

In [37]:
def adjust_weights(weights, max_weight):
    w = weights.copy()
    
    while (w > max_weight).any(axis=None):
        w[w > max_weight] = max_weight
        c = 1 - w.sum(axis=1)
        w_less = w[w < max_weight]
        w[(w < max_weight) & (w > 0)] += w_less.div(w_less.sum(axis=1), axis=0).mul(c, axis=0)
    
    return w

In [38]:
def remove_tiny_weights(weights, min_weight):
    w = weights.copy()
    
    w[w < min_weight] = 0
    w = w.div(w.sum(axis=1), axis=0)
    
    return w

In [39]:
adjusted_weights = adjust_weights(weights, max_weight=max_weight) 
adjusted_weights = remove_tiny_weights(adjusted_weights, min_weight=min_weight)
adjusted_weights = adjust_weights(adjusted_weights, max_weight=max_weight)

adjusted_weights = adjusted_weights.sort_values(adjusted_weights.last_valid_index(),ascending=False, axis = 1)

In [40]:
fig = px.line(adjusted_weights,
              labels={'value': 'weight, %', 'variable': ''})
fig.update_traces(
    hovertemplate="%{y}"
)
fig.update_yaxes(
    tickformat=".2%",
)
fig.update_xaxes(
    showspikes=True,
    spikethickness=2,
    spikedash="dot",
    spikecolor="#999999",
    spikemode="across",
)
fig.update_layout(
    hovermode="x",
    hoverdistance=100,  ## Distance to show hover label of data point
    spikedistance=1000,  ## Distance to show spike
    template='plotly_white',
    title='Weights with max constraint'
)

/Users/olivermehr/.local/share/virtualenvs/rebalancing-data-VGaZtODH/lib/python3.11/site-packages/_plotly_utils/basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



### Convert weights to the [1, 255] scale

On the contract side weights are integer numbers from the interval $[1, 255]$ with total sum $255$, so it's needed to convert retrieved weights to this format.

Note that one can't just round weights to integers after scaling, since it doesn't guarantee that their sum will be $255$. To fix that firstly floor function is applied to the weights and then $1$ is being added to the $k$ weights with largest fractional parts.

In [41]:
def convert_weights(weights):
    w_scaled = weights * 255
    w_res = np.floor(w_scaled).astype(int)    
    remainders = w_scaled - w_res    
    k = round(remainders.sum())
    w_below_max = w_res[w_res<76]
    for i in range(k):
        w_below_max[i] +=1
    for i in w_below_max.index:
        w_res[i] = w_below_max[i]
    return w_res

In [42]:
last_weights = adjusted_weights.iloc[-1].dropna()

In [43]:
converted_last_weights = convert_weights(last_weights)

/var/folders/4t/6j5ypx_s6r771shjr5dcct5w0000gn/T/ipykernel_36406/2561576563.py:8: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

/var/folders/4t/6j5ypx_s6r771shjr5dcct5w0000gn/T/ipykernel_36406/2561576563.py:8: FutureWarning:

Series.__setitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To set a value by position, use `ser.iloc[pos] = value`



## Summary table

In [44]:
pdi = pd.DataFrame()
pdi.index = chosen_tokens.index
pdi['name'] = chosen_tokens['name']
pdi['market_cap'] = marketcaps.iloc[-1].astype(int)
pdi['price'] = prices_data.iloc[-1]
pdi['weight'] = last_weights
pdi['weight_converted'] = converted_last_weights
pdi['address'] = [top_defi_tokens[symb.upper()]['address'] for symb in chosen_tokens['symbol']]

pdi = pdi[pdi['weight'] > 0]
pdi = pdi.sort_values("market_cap",ascending=False)
pdi

,name,market_cap,price,weight,weight_converted,address
id,,,,,,
uniswap,Uniswap,3215743125,4.267577,0.300000,76,0x1f9840a85d5af5bf1d1762f925bdaddc4201f984
lido-dao,Lido DAO,1370649227,1.531051,0.183126,47,0x5a98fcbea516cf06857215779fd812ca3bef1b32
maker,Maker,1028988103,1143.297295,0.137478,36,0x9f8f72aa9304c8b593d555f12ef6589cc3a579a2
aave,Aave,793183834,54.537095,0.105973,28,0x7fc66500c84a76ad7e9c93437bfc5ac33e2ddae9
rocket-pool,Rocket Pool,436252595,22.187698,0.058286,15,0xd33526068d116ce69f19a9ee46f0bd304f21a51f
frax-share,Frax Share,412588605,5.623089,0.055124,14,0x3432b6a60d23ca0dfca7761b7ab56459d9c964d0
curve-dao-token,Curve DAO,381607066,0.439264,0.050985,13,0xd533a949740bb3306d119cc777fa900ba034cd52
compound-governance-token,Compound,274358946,40.055970,0.036656,9,0xc00e94cb662c3520282e6f5717214004a7f26888
convex-finance,Convex Finance,216131587,2.697009,0.028876,7,0x4e3fbd56cd56c3e72c1403e103b45db9da5b9d2b


In [45]:
# Add assets and weights to database
db_assets = db_funcs.convert_to_sql_strings(list(pdi.index))
db_weights = list(pdi['weight'])
db_assets
db_funcs.insert_values(db_key_date,db_assets,db_weights,db_benchmark_table)

In [47]:
# Check if assets have been added to the registry
index_registry_address = "0x8f971223B4Bd9649702F10d306e469e6dbe3E3E9"
index_registry_instance  = web3_object.eth.contract(index_registry_address,abi=index_registry)
asset_role = "0x86d5cf0a6bdc8d859ba3bdc97043337c82a0e609035f378e419298b6a3e00ae6"

for i in pdi['address']:
    if index_registry_instance.functions.hasRole(asset_role,web3_object.to_checksum_address(i)).call() == False:
        print(i)

In [48]:
#Querying current contract to ascertain which assets will be dropped

pdi_token_address = '0x632806BF5c8f062932Dd121244c9fbe7becb8B48'
ia_contract = web3_object.eth.contract(address=pdi_token_address,abi=index_anatomy)
addresses, weights = ia_contract.functions.anatomy().call()

for address in addresses:
    address = address.lower()
    if address not in list(pdi['address']):
        symbol = tokens_by_addr[address]['symbol']
        data = cg_tokens_data.loc[symbol]
        pdi.loc[data['id']] = [data['name'],data['market_cap'],data['current_price'],0,0,address]
pdi

,name,market_cap,price,weight,weight_converted,address
id,,,,,,
uniswap,Uniswap,3215743125,4.267577,0.300000,76,0x1f9840a85d5af5bf1d1762f925bdaddc4201f984
lido-dao,Lido DAO,1370649227,1.531051,0.183126,47,0x5a98fcbea516cf06857215779fd812ca3bef1b32
maker,Maker,1028988103,1143.297295,0.137478,36,0x9f8f72aa9304c8b593d555f12ef6589cc3a579a2
aave,Aave,793183834,54.537095,0.105973,28,0x7fc66500c84a76ad7e9c93437bfc5ac33e2ddae9
rocket-pool,Rocket Pool,436252595,22.187698,0.058286,15,0xd33526068d116ce69f19a9ee46f0bd304f21a51f
frax-share,Frax Share,412588605,5.623089,0.055124,14,0x3432b6a60d23ca0dfca7761b7ab56459d9c964d0
curve-dao-token,Curve DAO,381607066,0.439264,0.050985,13,0xd533a949740bb3306d119cc777fa900ba034cd52
compound-governance-token,Compound,274358946,40.055970,0.036656,9,0xc00e94cb662c3520282e6f5717214004a7f26888
convex-finance,Convex Finance,216131587,2.697009,0.028876,7,0x4e3fbd56cd56c3e72c1403e103b45db9da5b9d2b


In [49]:
zipped_assets = list(zip(list(pdi['address']),list(pdi['weight_converted'])))
sorted_assets = sorted(zipped_assets,key = lambda x: int(x[0],base=0) )

asset_string = ''
for i in sorted_assets:
    asset_string += f'{i[0]},'
asset_string = asset_string[:-1]
print(asset_string)
print([x[1] for x in sorted_assets])

0x0bc529c00c6401aef6d220be8c6ea1667f6ad93e,0x1f9840a85d5af5bf1d1762f925bdaddc4201f984,0x3432b6a60d23ca0dfca7761b7ab56459d9c964d0,0x4e3fbd56cd56c3e72c1403e103b45db9da5b9d2b,0x5a98fcbea516cf06857215779fd812ca3bef1b32,0x6b3595068778dd592e39a122f4f5a5cf09c90fe2,0x7fc66500c84a76ad7e9c93437bfc5ac33e2ddae9,0x9f8f72aa9304c8b593d555f12ef6589cc3a579a2,0xba100000625a3754423978a60c9317c58a424e3d,0xc00e94cb662c3520282e6f5717214004a7f26888,0xc011a73ee8576fb46f5e1c5751ca3b9fe0af2a6f,0xd33526068d116ce69f19a9ee46f0bd304f21a51f,0xd533a949740bb3306d119cc777fa900ba034cd52
[6, 76, 14, 7, 47, 0, 28, 36, 4, 9, 0, 15, 13]
